# v7 -> v8


## Benefits of migrating:

- Ports, info and settings are stored in the GDS file, so you don't have to store them on separate files.
- Booleans are more robust when moving to integer based polygons, the slivers of less than 1nm disappear.


## Drawbacks of migrating

- As any changes in code, it can introduce undesired errors. It's recommended to have regression tests for all your components.


Major differences:

- xmin, xmax, ymin, ymax will be used for integers in Database Units (1nm by default), to set/get them in um You need to use `d` (decimal) Instance.dxmin which is in float. The same is true for `dmove`, `drotate`, `dsize_info`, `center`
- LayerMap is an Enum of integers
- New routing functions do not require starting ports to be in the same orientation and will monitor for self intersections. `get_route` -> `route_single`. `get_bundle` -> `route_bundle`
- All polygon points are snapped to grid.


Minor differences:

- replace `from gdsfactory.cell import cell` with `from gdsfactory import cell`
- we still don't have an equivalent


Improvements:

- gf.routing.route_single 


How to migrate?

We provide a migration script that migrated your files in place and that can help you migrate.

```bash
gf migrate -i --migration 7to8 <input_folder/file> or gf migrate --inplace --migration 7to8 <input_folder/file> 
```

## Use dxmin, dcenter, dmove, dsize_info

This is the biggest difference. By default all units are snap to grid.

In [ ]:
import gdsfactory as gf

c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
c

In [ ]:
print(ref.dxmax)

In [ ]:
print(ref.dsize_info.width)

## Use instance.drotate 

For setting arbitrary rotation angles you can use `Instance.drotate(30)` to rotate 30deg as an example.

In [ ]:
c = gf.Component()
ref = c << gf.components.bend_euler(radius=5)
ref.drotate(45)
c

## LayerMap 

In v7 or below, a LayerMap needs to be called

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab()
```

However in v8 it has a different type and does not need to be called 

```python

from gdsfactory.technology import LayerMap

class LayerMapFab(LayerMap):
    WG = (1, 0)

LAYER = LayerMapFab
```

See below:

In [ ]:
from gdsfactory.technology import LayerMap


class LayerMapFab(LayerMap):
    WG = (1, 0)


LAYER = LayerMapFab
type(LAYER)

In [ ]:
LAYER.WG

In [ ]:
tuple(LAYER.WG)

In [ ]:
str(LAYER.WG)

## Routing functions

Routing functions do no longer return the route Instances but they place the instances in a Component, so you have to pass a Component.

In [ ]:
c = gf.Component()
w = gf.components.straight(cross_section="rib")
top = c << w
bot = c << w
bot.dmove((0, -2))

p0 = top.ports["o2"]
p1 = bot.ports["o2"]

r = gf.routing.route_single(
    c,
    p0,
    p1,
    cross_section="rib",
)
c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)

ptop.dmovex(300)
ptop.dmovey(300)
routes = gf.routing.route_bundle_electrical(
    c,
    reversed(pbot.ports),
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[ptop.bbox(), pbot.bbox()],  # can avoid obstacles
)

c

In [ ]:
c = gf.Component()
columns = 2
ptop = c << gf.components.pad_array(columns=columns, port_orientation=270)
pbot = c << gf.components.pad_array(port_orientation=270, columns=columns)
ptop.dmovex(300)
ptop.dmovey(300)

obstacle = c << gf.c.rectangle(size=(300, 100), layer="M3")
obstacle.dymin = pbot.dymax - 10
obstacle.dxmin = pbot.dxmax - 10


routes = gf.routing.route_bundle_electrical(
    c,
    pbot.ports,
    ptop.ports,
    start_straight_length=100,
    separation=20,
    bboxes=[
        obstacle.bbox(),
        pbot.bbox(),
        ptop.bbox(),
    ],  # will ba able avoid obstacles soon
    sort_ports=True,
)

c